# Biopython Exam Script
Ian Poole

In [19]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

In [20]:
#fname = "dna.example.fasta"
fname = "dna2.fasta"
recs = SeqIO.parse(open(fname), 'fasta')
rec_list = list(recs)
print ("There are %d records" % len(rec_list))

There are 18 records


In [21]:
rec = rec_list[0]
print(rec.id)
print(rec.description)
print(rec.seq)
print(rec.letter_annotations)
dir(rec)
rec.name

gi|142022655|gb|EQ086233.1|91
gi|142022655|gb|EQ086233.1|91 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence
CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGGGAGGATCTCGGCGGCGCCAACTATGCGGTCTTTCGGCTCGAAAGCCAGTTCCAGACCTCCGACGGCGCGCTGACCGTGCCCGGCTCCGCATTCAGTTCGCAAGCCTACGTCGGGCTCGGCGGCGACTGGGGGACCGTGACGCTCGGGCGCCAGTTCGATTTCGTCGGCGATCTGATGCCGGCTTTCGCGATCGGCGCGAACACGCCGGCCGGCCTGCTCGCGTGGGGCTTGCCGGCGAATGCGTCGGCGGGCGGTGCGCTCGACAACCGCGTGTGGGGCGTCCAGGTGAACAATGCGGTGAAGTACGTGAGCCCGACGTTCGGCGGATTGTCGTTCGGCGGCCTGTGGGGCTTCGGCAACGTGCCCGGCACGGTCGCGCGCAGCAGCGTGCAAAGCGCGATGCTGTCCTACACGCAAGGCGCGTTCAGCGCCGCGCTCGCTTATTTCGGCCAGCACGATGTAACTGCCGGTGGCAATCTGCGCAATTTCTCGGGCGGTGCAGGCTACAACGTCGGGCAGTTCCGCGTCTTCGGCATGGTGTCGGACGTGCGGATCAGCGCCGCCGCGCCGCTGCGGGCCACGACCTATGACGGCGGCTTGACCTATGCGGTCACGCCGGCGTTGCAGCTCGGCGGCGGCTTCCAGTACCAGCAGCGCGGCGGCGACATCGGCTCGGCCAACCAGGTCACGTTGAGCGCCGACTATTCGCTGTCGAAGCGTACCGGCCTTTACGTGGTATTCGCACGCGGGCACGACAGTGCGTATGGCGCGCAGGTCGAGGCGGCGCTCGGCGGGG

'gi|142022655|gb|EQ086233.1|91'

## Min and max length sequences

In [22]:
for i, rec in enumerate(rec_list):
    print("%d id: %s.  length %d"%(i, rec.id, len(str(rec.seq))))


def max_or_min_summary(maxormin):
    len_list = [len(str(rec.seq)) for rec in rec_list]
    maxormin_len = max(len_list) if maxormin == 'max' else min(len_list)
    maxormin_len_indices = list(filter (lambda i: len_list[i] == maxormin_len, range(len(rec_list))))
    print()
    print("The %s sequence has length %d" % (maxormin, maxormin_len))
    print("They are:")
    for ix in maxormin_len_indices:
        print("%d: %s"%(ix, rec_list[ix].name))
        
max_or_min_summary("max")
print()
max_or_min_summary("min")

0 id: gi|142022655|gb|EQ086233.1|91.  length 4635
1 id: gi|142022655|gb|EQ086233.1|304.  length 1151
2 id: gi|142022655|gb|EQ086233.1|255.  length 4894
3 id: gi|142022655|gb|EQ086233.1|45.  length 3511
4 id: gi|142022655|gb|EQ086233.1|396.  length 4076
5 id: gi|142022655|gb|EQ086233.1|250.  length 2867
6 id: gi|142022655|gb|EQ086233.1|322.  length 442
7 id: gi|142022655|gb|EQ086233.1|88.  length 890
8 id: gi|142022655|gb|EQ086233.1|594.  length 967
9 id: gi|142022655|gb|EQ086233.1|293.  length 4338
10 id: gi|142022655|gb|EQ086233.1|75.  length 1352
11 id: gi|142022655|gb|EQ086233.1|454.  length 4564
12 id: gi|142022655|gb|EQ086233.1|16.  length 4804
13 id: gi|142022655|gb|EQ086233.1|584.  length 964
14 id: gi|142022655|gb|EQ086233.1|4.  length 2095
15 id: gi|142022655|gb|EQ086233.1|277.  length 1432
16 id: gi|142022655|gb|EQ086233.1|346.  length 115
17 id: gi|142022655|gb|EQ086233.1|527.  length 2646

The max sequence has length 4894
They are:
2: gi|142022655|gb|EQ086233.1|255


The mi

## Open reading frames for protien transcription

In [23]:
def find_rec_by_id(rec_list, identifier):
    recs = list(filter(lambda rec: rec.id==identifier, rec_list))
    assert len(recs) == 1
    return recs[0]

find_rec_by_id(rec_list, "gi|142022655|gb|EQ086233.1|91")

SeqRecord(seq=Seq('CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGG...GCC', SingleLetterAlphabet()), id='gi|142022655|gb|EQ086233.1|91', name='gi|142022655|gb|EQ086233.1|91', description='gi|142022655|gb|EQ086233.1|91 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence', dbxrefs=[])

In [24]:
def is_stop_codon(str):
    return str in ["TAA", "TAG", "TGA"]

assert ((is_stop_codon("TAA"), is_stop_codon("TAG"), is_stop_codon("TTT"))\
        == (True, True, False))
     

In [25]:
def index_from(s, c, i):
    ix = s[i:].find(c)
    if ix == -1:
        return -1
    else:
        return ix + i
index_from(".*..*.", "*", 2)

4

In [26]:
from collections import defaultdict

def extract_ORF_sequence(seq, interval):
    s,e = interval
    seq = Seq(str(seq[s:e]), seq.alphabet)
    print(seq)
    assert seq[:3] == "ATG"
    assert is_stop_codon(seq[-3:])
    return seq
    

def extract_ORF_indices_in_frame(seq, frame):
    """ Given a sequence, return a list of sub-sequences which start with
    a start codon (ATG) and end with a stop codon (TAA, TAG or TGA) in the
    current reading frame"""
    
    seq_str = str(seq)
    
    # Chop into codons
    codons = [seq_str[i:i+3] for i in range(frame, len(seq_str), 3)]
    # print(codons)
    
    # Convert to start S, end *, other . characters
    cdict = dict([("ATG", 'S'), ("TAA", "*"), ("TAG", "*"), ("TGA", "*")])
    scodons = [cdict[c] if c in cdict.keys() else '.' for c in codons]
    scodons = "".join(scodons)
    #print(scodons)
    
    # Now read of intervals
    intervals = []
    done = False
    start = 0
    end = 0
    while not done:
        start = index_from(scodons, 'S', end)
        if start != -1:
            end = index_from(scodons, '*', start)
            if end != -1:
                intervals += [(start, end)]
                start = end+1
            else:
                done = True
        else:
            done = True
            
    return [(s*3+frame, (e+1)*3+frame) for (s, e) in intervals]

def extract_ORF_indices(seq):
    intervals = []
    for frame in range(3):
        intervals += extract_ORF_indices_in_frame(seq, frame)
    return intervals
    
    
def test_extract_ORF_indices():
    def onetest(seq_str, expecting):
        print("Sequence = ", seq_str)
        seq = Seq(seq_str.replace(" ",""), IUPAC.unambiguous_dna)
        intervals = extract_ORF_indices(seq)
        for (s,e) in intervals:
            print((s,e))
            print(extract_ORF_sequence(seq, (s,e)))
        assert intervals == expecting
    
    onetest("CCC   ATG TAA  ATG GGG TAG   GGGG", [(3,9), (9,18)])
    onetest("ATG CGC TTT TAG", [(0, 12)])
    onetest("T ATG CGC TTT TAG", [(1, 13)])
    onetest("ATG A ATG TAG", [(4, 10)])
    onetest("AT ATG G TAG", [])
    onetest("A  ATG CGC TAG  AAA  ATG TAG  G", [(1, 10), (13, 19)])
        
test_extract_ORF_indices()

Sequence =  CCC   ATG TAA  ATG GGG TAG   GGGG
(3, 9)
ATGTAA
ATGTAA
(9, 18)
ATGGGGTAG
ATGGGGTAG
Sequence =  ATG CGC TTT TAG
(0, 12)
ATGCGCTTTTAG
ATGCGCTTTTAG
Sequence =  T ATG CGC TTT TAG
(1, 13)
ATGCGCTTTTAG
ATGCGCTTTTAG
Sequence =  ATG A ATG TAG
(4, 10)
ATGTAG
ATGTAG
Sequence =  AT ATG G TAG
Sequence =  A  ATG CGC TAG  AAA  ATG TAG  G
(1, 10)
ATGCGCTAG
ATGCGCTAG
(13, 19)
ATGTAG
ATGTAG


In [27]:
max_frame_len = 0
max_frame_rec = None
max_frame_interval = None

for rec in [find_rec_by_id(rec_list, "gi|142022655|gb|EQ086233.1|16")]: # rec_list:
    intervals = extract_ORF_indices(rec.seq)
    for (s,e) in intervals:
        frame_len = e-s
        if frame_len > max_frame_len:
            max_frame_len = frame_len
            max_frame_rec = rec
            max_frame_interval = s,e

print("Longest frame:")
print("Record id:%s" % max_frame_rec.id)
print("Codon length: %d" % max_frame_len)
print("Interval:", max_frame_interval)
seq = extract_ORF_sequence(max_frame_rec.seq, max_frame_interval)
print("Sequence:")
print(seq)
print("Protein:")
print(seq.translate())

Longest frame:
Record id:gi|142022655|gb|EQ086233.1|16
Codon length: 1644
Interval: (1439, 3083)
ATGCGGGCCATCCTGCATCGCCGCCTTTCGTTCCACCCGGGCCGGCATCGAGTGATGCCGGCGTTGACGTTTTCGTGGAGTGAGTCAGATGAATCACGCAGCGAATCCCGCCGATCCCGATCGCGCCGCGGCGCAGGGCGGCAGCCTGTACAACGACGATCTCGCGCCGACGACGCCGGCGCAGCGCACGTGGAAGTGGTATCACTTCGCGGCGCTGTGGGTCGGGATGGTGATGAACATCGCGTCGTACATGCTCGCGGCCGGGCTGATCCAGGAAGGCATGTCGCCGTGGCAGGCGGTGACGACGGTGCTGCTCGGCAACCTGATCGTGCTCGTGCCGATGCTGCTGATCGGCCATGCGGGCGCGAAGCACGGGATTCCGTACGCGGTGCTCGTGCGCGCGTCGTTCGGCACGCAGGGGGCGAAGCTGCCGGCGCTGCTGCGCGCGATCGTCGCGTGCGGCTGGTACGGGATCCAGACCTGGCTCGGCGGCAGCGCGATCTATACGCTGCTGAACATCCTGACCGGCAACGCGCTGCATGGCGCCGCGCTGCCGGTCATCGGCATCGGGTTCGGGCAGCTCGCATGCTTCCTCGTGTTCTGGGCGCTGCAGCTCTACTTCATCTGGCATGGCACCGATTCGATCCGCTGGCTCGAAAGCTGGTCGGCGCCGATCAAGGTCGTGATGTGCGTGGCGCTGGTGTGGTGGGCAACGTCGAAGGCGGGCGGCTTCGGCACGATGCTGTCGGCGCCGTCGCAGTTTGCCGCAGGCGGCAAGAAAGCCGGGCTGTTCTGGGCGACCTTCTGGCCGGGGCTGACCGCGATGGTCGGCTTCTGGGCGACGCTCGCGCTGAACATCCCCGACTTCACGCGCTTCGCGCATTCGCAGCGCGACCAGGTGAT

In [28]:
exam_id = "gi|142022655|gb|EQ086233.1|16"
rec = find_rec_by_id(rec_list, exam_id)
extract_ORF_indices(rec.seq)


[(264, 330),
 (576, 603),
 (1272, 1494),
 (1527, 3036),
 (4719, 4788),
 (460, 856),
 (904, 1522),
 (1795, 1954),
 (1978, 2125),
 (3070, 4528),
 (110, 1427),
 (1439, 3083),
 (3104, 3236),
 (3401, 3461),
 (3629, 3689),
 (3797, 3890),
 (4148, 4301),
 (4577, 4667)]

## DNA repeat sequences (slow way!)
Finding repeats of a given (short) length

In [29]:
import itertools

def generate_all_substrings(n):
    return ["".join(t) for t in itertools.product("CGAT", repeat=n)]

generate_all_substrings(2)
    

['CC',
 'CG',
 'CA',
 'CT',
 'GC',
 'GG',
 'GA',
 'GT',
 'AC',
 'AG',
 'AA',
 'AT',
 'TC',
 'TG',
 'TA',
 'TT']

In [30]:
import re

def count_given_substrings_in_all_sequences(substr, with_overlap):
    if with_overlap:
        c = sum([len(re.findall("(?=%s)"%substr, str(rec.seq))) for rec in rec_list])
    else:
        c = sum([rec.seq.count(substr) for rec in rec_list])
    return c

def test_count_given_substrings_in_all_sequences():
    substr = "AAA"
    print("With overlap: ", count_given_substrings_in_all_sequences(substr, True))
    print("Without overlap: ", count_given_substrings_in_all_sequences(substr, False))
test_count_given_substrings_in_all_sequences()  

With overlap:  222
Without overlap:  171


In [31]:

def count_all_substrings_of_length_n(n, with_overlap):
    all_substrings = generate_all_substrings(n)
    counts = [(substr, count_given_substrings_in_all_sequences(substr, with_overlap))
                 for substr in all_substrings]
    
    sorted_counts = sorted(counts, key=lambda t: -t[1])
    for (substr,c) in sorted_counts:
        if c != 0:
            print (substr, c)
        
count_all_substrings_of_length_n(6, True)

GCGCGC 153
CGCGCG 151
GCCGCG 147
GCGCCG 135
CGCGGC 131
CGGCCG 125
CGCCGC 121
GCCGGC 119
GCGCGG 115
CGGCGC 109
GCGGCG 106
CCGCGC 105
CGCCGA 105
GCGACG 104
CGCGCC 103
GCGCGA 102
CGCGAC 100
GGCGCG 99
CGACGC 97
CGACGA 96
TCGCGC 96
CGGCGA 95
CCGGCG 94
CGCGCA 94
TCGGCG 93
CGATCG 90
GCGGCC 90
TGCGCG 90
GGCCGC 89
TCGTCG 89
CGAGCG 88
GCCGCC 87
TCGCCG 87
GCCGAC 84
CGTCGC 83
GCGCTG 83
CGCCGG 82
CGCTCG 82
GGCGGC 81
CCGCCG 80
CGCGAT 80
ATCGCG 80
CCGGCC 79
GCGATC 79
CCGCGA 78
GCGTCG 77
TCGACG 77
GATCGC 76
CCGACG 75
CGATGC 74
AGCGCG 74
CGGCGG 73
GTCGCG 73
CGGCGT 72
CGTCGA 72
CGTGCG 71
CGCGCT 70
GGCCGG 70
CGCTGC 69
TCGCGG 68
CTCGCG 67
GTCGGC 67
CGACGG 65
GACGGC 65
TCGCGA 65
CGCGGT 64
CGGGCG 64
GCCGAT 64
GCGAGC 64
ACGCCG 64
ACGCGC 64
TGCTGC 64
TGGCCG 64
CGCCCG 63
CGCGAG 63
CGGTCG 63
CAGCGC 63
CTGCCG 63
GCTCGC 63
GGCCGA 63
TCGAGC 62
TGCCGC 62
CGCGTG 61
CGTCGT 61
GCACGC 61
CGCGAA 60
GCAGCG 60
GCATCG 60
GCGGGC 59
GCTCGA 59
TGCCGG 59
CACGCG 58
GCGATG 58
ACGCGA 58
CGCCGT 57
GCCCGC 57
GCGGCA 57
GATGCG 57
CGC

GGATCC 9
GGATGG 9
GGTCAG 9
GAGGGC 9
GAGAAG 9
GAACCC 9
GAACCT 9
GATGGA 9
GATGGT 9
GATGAG 9
GATACC 9
GTCCAT 9
GTGGCT 9
GTGGAG 9
GTGTGC 9
GTATTC 9
GTTTCA 9
ACCCCG 9
ACCCGT 9
ACCGAG 9
ACCAGA 9
ACGCCA 9
ACGCAA 9
ACGGGG 9
ACGTCA 9
ACGTAC 9
ACAGCT 9
ACAGGT 9
ACATGC 9
ACTCGA 9
ACTCGT 9
ACTGCG 9
AGCGTT 9
AGCAAC 9
AGGTGG 9
AGGTTC 9
AGACGG 9
AGAACA 9
AGTCGG 9
AGTGCG 9
AGTTCA 9
AACCGT 9
AACCAG 9
AAACCC 9
AAACGA 9
ATCCGT 9
ATCCAC 9
ATGATG 9
ATGTTC 9
ATAGCC 9
ATATCG 9
TCCGGG 9
TCGGTT 9
TCACCT 9
TCAGGA 9
TCAGGT 9
TCTGGA 9
TCTGGT 9
TGCGAG 9
TGGCCA 9
TGGAGC 9
TGGTGT 9
TGGTAC 9
TGAAGT 9
TGTTCT 9
TACGCC 9
TATCGA 9
TTCCGT 9
TTCGCT 9
TTGCGA 9
TTGGTC 9
TTGATC 9
CCCGGG 8
CCCGTA 8
CCGGGA 8
CCGTCT 8
CCACCC 8
CCACAC 8
CCTCCA 8
CCTGTG 8
CCTTTC 8
CGGGGA 8
CGAATT 8
CGTGTG 8
CGTATG 8
CACCTT 8
CACACC 8
CAGAGC 8
CAATCC 8
CAATGC 8
CATCCC 8
CATGTT 8
CATTGA 8
CATTTC 8
CTCCTC 8
CTGGCA 8
CTGGAG 8
CTGGAT 8
CTGTCC 8
CTGTTG 8
CTACGA 8
CTACGT 8
CTTGCT 8
CTTGAC 8
CTTTCG 8
GCCCCC 8
GCCAAC 8
GCCTTG 8
GCGTCT 8
GCACAT 8
GCAGAG 8
G

ATACAC 2
ATACTG 2
ATAGCT 2
ATAGTT 2
ATAACG 2
ATAAAG 2
ATAATA 2
ATATGG 2
ATATGT 2
ATATTC 2
ATTCAC 2
ATTCAA 2
ATTGGT 2
ATTGAG 2
ATTGTT 2
ATTACT 2
ATTAAA 2
ATTTAT 2
TCCCAC 2
TCCCAT 2
TCCATT 2
TCGCTA 2
TCACAG 2
TCACAA 2
TCAGTC 2
TCAGTA 2
TCAAGA 2
TCTCAG 2
TCTCAA 2
TCTCTC 2
TCTCTG 2
TCTCTT 2
TCTGCC 2
TCTGAA 2
TCTGAT 2
TCTACC 2
TCTATA 2
TCTTGT 2
TCTTTT 2
TGCACT 2
TGCAAA 2
TGGCTA 2
TGGGGG 2
TGGACT 2
TGAGGA 2
TGAGGT 2
TGAGAC 2
TGAGTT 2
TGAAAA 2
TGAATA 2
TGATAC 2
TGTCAG 2
TGTCTT 2
TGTGAT 2
TGTGTG 2
TGTGTA 2
TGTACC 2
TGTAGA 2
TGTAAT 2
TACCTC 2
TACGAG 2
TACAGA 2
TACAAT 2
TACTCC 2
TACTGT 2
TAGCCA 2
TAGGCT 2
TAGAAC 2
TAGAAG 2
TAGAAA 2
TAGTCC 2
TAGTGA 2
TAACGC 2
TAAATC 2
TAATGC 2
TAATAT 2
TATCCC 2
TATCAG 2
TATGGT 2
TATGAC 2
TATGAA 2
TATAGC 2
TATTCC 2
TATTGA 2
TATTAC 2
TATTTG 2
TTCCAA 2
TTCGGA 2
TTCACT 2
TTCATA 2
TTCATT 2
TTCTTG 2
TTGGGC 2
TTGTCA 2
TTGTCT 2
TTGTGT 2
TTGTTT 2
TTACCA 2
TTACTG 2
TTAAAC 2
TTATCC 2
TTATTT 2
TTTGCA 2
TTTGCT 2
TTTGGC 2
TTTGAA 2
TTTGTG 2
TTTTAC 2
CCCCCT 1
CCCCAA 1
CCCAAT 1
C

## Repeat sequeces - the proper way
Use a Counter (dictionary) to hold sequences

In [32]:
from collections import Counter

cnt = Counter()
cnt['Fred'] += 1
cnt['Fred'] += 1
print (cnt['Fred'], cnt['Harry'])

2 0


In [33]:
from Bio.Alphabet import SingleLetterAlphabet
def count_all_substrings_of_length_n_Counter(seq, n, init_cnt=None):
    if init_cnt is None:
        cnt = Counter()
    else:
        cnt = init_cnt
    seq_str = str(seq)
    for i in range(0, len(seq_str)-n+1):
        substr = seq_str[i:i+n]
        cnt[substr] += 1
    return cnt

def test_count_all_substrings_of_length_n_Counter():
    def test_one(s, n, expecting):
        seq = Seq(s, SingleLetterAlphabet())
        cnt_result = count_all_substrings_of_length_n_Counter(seq, n)
        print(str(seq), ' : ', cnt_result)
        assert cnt_result == expecting
        
    test_one('A', 1, Counter({'A':1}))
    test_one('AA', 1, Counter({'A':2}))
    test_one('AAAA', 2, Counter({'AA': 3}))
    test_one('AGAGA', 3, Counter({'AGA':2, 'GAG':1}))
    test_one('AGAGAT', 3, Counter({'AGA':2, 'GAG':1, 'GAT':1}))
test_count_all_substrings_of_length_n_Counter()


A  :  Counter({'A': 1})
AA  :  Counter({'A': 2})
AAAA  :  Counter({'AA': 3})
AGAGA  :  Counter({'AGA': 2, 'GAG': 1})
AGAGAT  :  Counter({'AGA': 2, 'GAG': 1, 'GAT': 1})


In [34]:
def count_all_substrings_of_length_n_in_all_records_Counter(n):
    cnt = Counter()
    for rec in rec_list:
        count_all_substrings_of_length_n_Counter(rec.seq, n, cnt)
        
    return cnt
    
    
cnt = count_all_substrings_of_length_n_in_all_records_Counter(6)

print(cnt.most_common(5))
print('******')

for key in cnt.keys():
        if cnt[key] > 1:
            print(key, ' : ', cnt[key])
        

[('GCGCGC', 153), ('CGCGCG', 151), ('GCCGCG', 147), ('GCGCCG', 135), ('CGCGGC', 131)]
******
CTCGCG  :  67
TCGCGT  :  40
CGCGTT  :  38
GCGTTG  :  29
CGTTGC  :  36
GTTGCA  :  17
TTGCAG  :  17
TGCAGG  :  27
GCAGGC  :  41
CAGGCC  :  32
AGGCCG  :  45
GGCCGG  :  70
GCCGGC  :  119
CCGGCG  :  94
CGGCGT  :  72
GGCGTG  :  43
GCGTGT  :  32
CGTGTC  :  33
GTGTCG  :  45
TGTCGC  :  26
GTCGCG  :  73
TCGCGC  :  96
CGCGCA  :  94
GCGCAA  :  26
CGCAAC  :  29
GCAACG  :  28
CAACGA  :  13
AACGAC  :  15
ACGACG  :  50
CGACGT  :  48
GACGTG  :  24
ACGTGT  :  7
CGTGTG  :  8
GTGTGG  :  8
TGTGGG  :  12
GTGGGG  :  11
TGGGGC  :  13
GGGGCC  :  8
GGGCCT  :  10
GGCCTG  :  25
GCCTGA  :  16
CCTGAC  :  10
CTGACG  :  18
TGACGG  :  20
GACGGG  :  14
ACGGGC  :  20
CGGGCA  :  38
GGGCAG  :  23
GGCAGG  :  22
GCAGGG  :  14
CAGGGA  :  2
AGGGAG  :  2
GGGAGG  :  4
GGAGGA  :  5
GAGGAT  :  5
AGGATC  :  14
GGATCT  :  10
GATCTC  :  28
ATCTCG  :  31
TCTCGG  :  14
CTCGGC  :  53
TCGGCG  :  93
CGGCGG  :  73
GGCGGC  :  81
GCGGCG  :  106
CGGC

TTTGTC  :  3
TCGTTA  :  3
AGTAAA  :  2
GTAAAA  :  3
TAAAAC  :  3
AAAACG  :  6
AAACGA  :  9
AACGAA  :  8
ACGAAA  :  11
AAATAT  :  3
AATATG  :  4
ATATGA  :  3
TATGAA  :  2
ATGAAT  :  8
TGAATT  :  3
AATTAC  :  3
ATTACG  :  5
TTACGA  :  3
TACGAA  :  8
ACGAAG  :  27
CGAAGA  :  14
AGAAAA  :  5
GAAAAG  :  6
AAAAGA  :  6
AAAGAG  :  2
AAGAGG  :  4
AGAGGC  :  4
AGGCCC  :  7
ATCGGG  :  19
CGGACC  :  16
GACCGG  :  19
CGGATG  :  20
GGATGA  :  17
GATGAC  :  17
GACGAA  :  36
ACGAAC  :  19
GCCGGG  :  41
CCGGGC  :  42
CGTGGC  :  37
GCGTTA  :  4
CGTTAC  :  9
GTTACA  :  2
ACAGCA  :  13
CAGCAT  :  18
GCATCC  :  23
ATCCGT  :  9
TCCGTC  :  11
GTCGCA  :  28
TCGCAG  :  18
CGTCGT  :  61
TCGTTT  :  6
CGTTTT  :  5
GTTTTC  :  4
TTCCGG  :  19
TCCGGG  :  9
CCGGGT  :  13
CGGGTT  :  13
AAGAAC  :  11
CGGTGT  :  18
GGTGTT  :  8
TGTTTC  :  6
GTTTCA  :  9
GCCAAG  :  12
CCAAGC  :  10
CAAGCA  :  12
AAGCAC  :  10
AGCACA  :  5
GCACAT  :  8
CATGCA  :  12
ATGCAG  :  16
CGATGA  :  29
GATGAG  :  9
ATGAGC  :  10
GAGCAA  :  7
AGCA

TTTGCC  :  6
AGGTGG  :  9
GGTGGT  :  11
GTGACA  :  4
GAGGAG  :  4
AGGAGA  :  4
GCGAAA  :  16
CGAAAC  :  17
GGAGCA  :  8
TCCGGA  :  18
GGGCAA  :  6
GGCAAG  :  21
CAGACG  :  9
AGACGG  :  9
GGCCCC  :  5
ATCTCA  :  3
CGGTAC  :  11
GGTACC  :  3
GTACCC  :  3
TACCCG  :  6
CCGGGG  :  7
AGCAGA  :  10
GATGTG  :  11
TGTGGA  :  8
CTGGGT  :  3
GGGTGA  :  7
GGTGAG  :  6
GCCCAA  :  5
CCCAAA  :  2
AAAACC  :  4
AAACCG  :  10
CAAGGA  :  9
AAGGAC  :  8
AGGACC  :  6
GGGGGG  :  3
GGGGAG  :  3
GGGAGC  :  3
AGCAGT  :  8
GTTCGT  :  14
GTACCT  :  5
TACCTG  :  8
GACGGA  :  15
ACGATA  :  5
TACCGA  :  7
GAGGTG  :  7
TGGAGA  :  7
AGGCCA  :  12
ATCTGT  :  6
CTGTGC  :  4
GGGGGT  :  2
GGGTGG  :  10
CTCGGA  :  4
CGGAGT  :  6
GGAGTA  :  3
GAGTAC  :  6
TACCTC  :  2
TGGAAG  :  6
GGAAGC  :  8
CGTCTA  :  4
GTCTAT  :  8
CTATCG  :  3
TGCCCC  :  6
GTGTAC  :  7
GGTAGG  :  2
TGTCAG  :  2
GCACCT  :  12
ACCTGC  :  21
GTAACG  :  6
TAACGT  :  3
CGGCTA  :  9
GGCTAT  :  6
GCTATC  :  5
CAAACC  :  4
GACATA  :  3
ACATAT  :  3
CATATT  : 

In [35]:
c1 = Counter({'a':1, 'b':2})
c2 = Counter({'b':2, 'a':1})
c1 == c2

True

In [36]:
c1

Counter({'a': 1, 'b': 2})